# Introduction to the Python control system

This is the introductory example explaining the connection setup between Python and the **Term** Terminal Software running the **Thales** Workstation Software.

In [73]:
import sys
from thales_remote.connection import ThalesRemoteConnection
from thales_remote.script_wrapper import PotentiostatMode,ThalesRemoteScriptWrapper
import math
import cmath

from jupyter_utils import executionInNotebook, notebookCodeToPython

# Utility functions

First, two utility functions are declared.  
With the first function the complex impedance is output to the console. The second function measures an impedance spectrum from individual impedance measuring points.

In [74]:
def printImpedance(impedance):
    print("Impedance: " + str(abs(impedance)) + " ohm, " + str(cmath.phase(impedance)) + " rad")
    return

def spectrum(scriptHandle, lower_frequency, upper_frequency, number_of_points):
    log_lower_frequency = math.log(lower_frequency)
    log_upper_frequency = math.log(upper_frequency)
    log_interval_spacing = (log_upper_frequency - log_lower_frequency) / (number_of_points - 1)
    
    for i in range(number_of_points):
        current_frequency = math.exp(log_lower_frequency + log_interval_spacing * i)
        print("Frequency: " + str(current_frequency))
        printImpedance(scriptHandle.getImpedance(current_frequency))
        
    return

# Connect to the Term software

Before calling the following commands, the Term software must be started and the Thales start screen must be displayed, in which methods can be selected.

In this case the Term runs on the same computer as Python, so **"localhost"** is chosen here as IP address, but you could also specify the IP address of the computer in the network, on which the Term software is running and to which the workstation is connected via USB.

In [75]:
TARGET_HOST = "localhost"

if __name__ == "__main__":
    zenniumConnection = ThalesRemoteConnection()
    connectionSuccessful = zenniumConnection.connectToTerm(TARGET_HOST, "ScriptRemote")
    if connectionSuccessful:
        print("connection successfull")
    else:
        print("connection not possible")
        sys.exit()

connection successfull


# Initialize the application and start the remote software in Thales

If the connection to the term is successfully established, the [ThalesRemoteConnection]([**ThalesRemoteScriptWrapper**](http://zahner.de/documentation/thales_remote/script_wrapper.html#thales_remote.script_wrapper.ThalesRemoteScriptWrapper)) object that manages the connection to the Thales software. This object is passed to the constructor of a new [ThalesRemoteScriptWrapper](http://zahner.de/documentation/thales_remote/script_wrapper.html#thales_remote.script_wrapper.ThalesRemoteScriptWrapper) object.

The [ThalesRemoteScriptWrapper](http://zahner.de/documentation/thales_remote/script_wrapper.html#thales_remote.script_wrapper.ThalesRemoteScriptWrapper) class implements the commands as methods, which are available via [Remote2](http://zahner.de/pdf/Remote2.pdf).

In [ ]:
    zahnerZennium = ThalesRemoteScriptWrapper(zenniumConnection)
    zahnerZennium.forceThalesIntoRemoteScript()

# DC measurement

## Initialization of the potentiostat

First, a voltage of 1.0 V is output potentiostatically as an example. For this, potentiostatic is set first. Then the potential is set and the potentiostat is switched on.

In [ ]:
    zahnerZennium.setPotentiostatMode(PotentiostatMode.POTMODE_POTENTIOSTATIC)
    zahnerZennium.setPotential(1.0)
    zahnerZennium.enablePotentiostat()

## Execute the measurement

After the potentiostat is switched on, voltage and current are measured several times in a loop.

In [78]:
    for i in range(5):
        print("Potential: " + str(zahnerZennium.getPotential()))
        print("Current: " + str(zahnerZennium.getCurrent()))

Potential: 0.999819
Current: 0.000999695
Potential: 0.999821
Current: 0.0009996858
Potential: 0.9998204
Current: 0.0009996853
Potential: 0.9998235
Current: 0.0009996788
Potential: 0.9998211
Current: 0.0009996682


# Impedance measurement

## Parameterization of the impedance measurement

For the impedance measurement, the measuring frequency, the measuring amplitude and the number of periods to be averaged are now set.

In [ ]:
    zahnerZennium.setFrequency(2000)
    zahnerZennium.setAmplitude(10e-3)
    zahnerZennium.setNumberOfPeriods(3)

## Execute the measurement

Since the potentiostat is still switched on from the DC measurement, the impedance spectrum is now measured at the set DC potential of 1 V.

<div class="alert alert-block alert-info">
<b>Note:</b> If the potentiostat is set to potentiostatic before the impedance measurement and is switched off, the measurement is performed at the open circuit voltage/potential.
</div>
    

In [80]:
    printImpedance(zahnerZennium.getImpedance())
    printImpedance(zahnerZennium.getImpedance(2000))
    printImpedance(zahnerZennium.getImpedance(2000, 10e-3, 3))

    spectrum(zahnerZennium, 1000, 2e5, 10)
    
    zahnerZennium.disablePotentiostat()

Impedance: 998.9000535233743 ohm, 0.00032736008441190895 rad
Impedance: 999.4001470244089 ohm, 0.0005424254020747291 rad
Impedance: 999.8000279007997 ohm, 0.00023624724505468624 rad
Frequency: 999.9999999999998
Impedance: 999.4003471527313 ohm, 0.0008334999070430484 rad
Frequency: 1801.64823065441
Impedance: 1000.0000260832796 ohm, -0.00022839999602838602 rad
Frequency: 3245.936347020167
Impedance: 999.9000349043098 ohm, -0.0002642264164932222 rad
Frequency: 5848.035476425735
Impedance: 1000.0000059405 ohm, 0.00010899999956832366 rad
Frequency: 10536.102768906649
Impedance: 998.5000447377106 ohm, 0.00029934901459376426 rad
Frequency: 18982.350911593698
Impedance: 999.6000291244944 ohm, 0.00024139655393453876 rad
Frequency: 34199.51893353398
Impedance: 1000.0004020353641 ohm, -0.000896699759663327 rad
Frequency: 61615.50277583351
Impedance: 999.3004679686735 ohm, 0.000967777142073199 rad
Frequency: 111009.46155696237
Impedance: 999.9098222459863 ohm, -0.0044324142172560595 rad
Frequency

'OK\r'

# Close the connection

In [81]:
    zenniumConnection.disconnectFromTerm()
    print("finish")

finish


# Deployment of the source code

**The following instruction is not needed by the user.**

It automatically extracts the pure python code from the jupyter notebook to provide it to the user. Thus the user does not need jupyter itself and does not have to copy the code manually.

The source code is saved in a .py file with the same name as the notebook.

In [82]:
    if executionInNotebook() == True:
        notebookCodeToPython("BasicIntroduction.ipynb")